## Ch09. 함수 II
1. 중첩함수
2. 제너레이터
3. 재귀함수

### 9-1. 중첩함수
- 캡슐화 목적
    - 변수 범위를 제한할 수 있다. 
    - 책임, 관리 명확해짐

In [65]:
def outer(a, b): # 외부함수 
    def inner(c, d): # 내부함수, inner 함수는 밖에서 사용할 수 없음
        return c + d # c, d 변수도 이 안에서만 사용
    return inner(a, b) # 함수가 반환값이 될 수 있다. 

outer(2, 2)

4

In [66]:
inner(2, 2) # NameError => 선언한 적 없다. 중첩함수 쓰는 이유(캡슐화와 비슷)

NameError: name 'inner' is not defined

In [67]:
def knights(saying):
    def inner(quote):
        return f'we are the knights who say: {quote}'
    return inner(saying)

#knights("Ni")
case1 = knights('Ni!') # 변수로 받은 것이 의미 있으려면 
                       # return inner(saying->이부분이 없어야 함)

In [68]:
case1

'we are the knights who say: Ni!'

#### 1) 클로저
- 자신을 둘러싼 scope(name space)의 상태값을 기억하는 함수 (아래 코드의 1) 부분)
- 메모리 효율적 사용 (함수 호출 시 꺼내쓸 수 있다)
    - 호출하기 전까지는 메모리에 올라가지 않음
- 조건
    - 중첩함수여야 함
    - 외부함수의 상태값 참조해야 함
    - 외부함수가 내부함수를 반환해야 함

In [69]:
# 위 중첩함수와 다른 점: 1) & 2)
    # -> 함수의 모습을 다양하게, 함수를 변수로 받음, 함수 재활용 가능

def multiply(x):
    def inner(y):
        return x * y # 1) 외부함수의 인자 참조
    return inner     # 2) 함수 실행되기 전 상태를 리턴 (inner(), inner(x); 괄호를 안썼기 때문)

multiply # multiply의 반환값은 함수

<function __main__.multiply(x)>

In [70]:
# 2) 밖에서 함수 선언
m5 = multiply(5) # x = 5 
m6 = multiply(6) # x = 6

In [71]:
m5(6)

30

- 메모리가 효율적인 이유

In [72]:
del(multiply) # multiply function 삭제

In [73]:
multiply # 삭제되었음

NameError: name 'multiply' is not defined

In [74]:
m5(10) # inner의 상태 계속 유지, 메모리는 안쓰고 있음

50

#### 2) 데코레이터
- 메인 함수에 또 다른 함수를 데코레이터로 선언하여 사용할 수 있음
- 목적: 
    - 재사용, 가독성, 직관적임

In [75]:
def document_it(func):
    def new_func(*args, **kargs):
        print('arguments: ', *args)
        print('key arguments: ', kargs)
        return func(*args, **kargs)
    return new_func

@document_it
def add(a, b):
    return a + b

@document_it
def subtract(a, b):
    return a - b

@document_it
def square(a):
    return a**2

In [76]:
add(3, 4)

arguments:  3 4
key arguments:  {}


7

In [77]:
# add에 활용할 중첩함수 만들기
    # 결과값의 제곱값을 반환하는 클로저 함수 만들기
    
def square(func):
    def new_func(*args): # args -> 2개 이상이 들어와도 적용 가능
        result = func(*args)
        return result * result
    return new_func

@square
def add(a, b):
    return a + b

In [78]:
add(3, 4)

49

#### 3) scope: global, local, nonlocal
- 내부함수는 외부함수의 인자를 "참조"만 할 수 있다. (읽기만 가능)
- nonlocal 예약어를 사용

In [79]:
# 전역(global) 
def square(func):
    # 지역(local)
    def new_func(*args):
        # 지역 내의 지역?
        result = func(*args)
        return result * result
    return new_func

In [80]:
z = 3 # 전역변수
def outer(x): # x값 "참조": x 출력이나 반환 가능하지만 연산을 할 수는 없다. 
    y = 4 # 지역변수
    def inner():
        # 여기선 x = 10
        x = 1000 
        return x # 여기선 1000
    #return x -> outer(10) = 10
    return inner()
    
outer(10)

1000

In [81]:
z = 3 
def outer(x): 
    y = 4 
    def inner():
        # 지정안하면 (UnboundLocalError: local variable 'x' referenced before assignment)
        nonlocal x 
        # 가장 가까운 지역에 있는 변수를 제일 먼저 인식 (전역/외부함수/내부함수 3개의 지역)
        # 이 x가 outer(x)의 x 지칭.
        x += 1 
        return x # 여기선 1000
    return inner()
    
outer(10)

11

In [82]:
def my_func(nums:list): # 가변인자 -> 리턴 없이도 리턴 얻음
    # 가변인자(ex.list)를 하는 것 지양
    # 꼭 필요하다면 문서화 => 사용자가 알 수 있게
    nums.append(sum(nums))
    
a = [1, 2, 3]
my_func(a)

In [83]:
my_func(a)
my_func(a)
my_func(a)

In [84]:
a

[1, 2, 3, 6, 12, 24, 48]

#### <실습>
1. 함수 : 차 속도, 제한 속도를 비교해서 true/false 반환

2. 데코레이터 함수
- 함수를 얹을 중첩함수
- 만약 제한 속도를 초과했다면 얼마나 초과했는지 프린트하는 함수
- 예: 100, 80
- "20 km/h 초과"


In [87]:
# 내 코드
def excess(func):
    def new_func(*args):
        if not func(*args):
            return f'{args[0]-args[1]} km/h 초과'
        else:
            return f'제한 속도 준수'
    return new_func

@excess
def compare(a, b):
    return a < b

In [86]:
compare(70, 80)

'제한 속도 준수'

In [88]:
# 모범 코드
def calc_speed(func):
    def inner(speed, limit):
        if func(speed, limit):
            return f'초과 속도: {speed - limit} km/h'
        else:
            return f'정상 속도'
    return inner

In [89]:
@calc_speed
def is_speeding(speed, limit):
    return speed > limit

is_speeding(100, 80)

'초과 속도: 20 km/h'

### 9-2. 제너레이터
- return -> yield
- 순회의 리턴값을 하나씩 반환
- 시퀀스를 생성하는 객체
- 메모리 효율성 증대
    - 한번 사용되고 메모리에서 사라짐

In [ ]:
# 형태
def ...():
    for i in range(5):
        yield i

In [91]:
names = 'Kevin Michael Juliette Laura'.split()

def printing(name_list:list):
    for name in name_list:
        yield name
        
name_list = printing(names)

In [92]:
for i in printing(names):
    print(i)

Kevin
Michael
Juliette
Laura


In [93]:
# 한번만 쓸 수 있다.
for i in name_list:
    print(i)

Kevin
Michael
Juliette
Laura


#### <실습: range 함수 구현하기>
`
def my_range(start, end, step):
    # 작성하기
    yield
`

In [95]:
def my_range(start, end, step=1):
    while start < end:
        yield start
        start += step

ranger = my_range(1, 10, 2)

- list comprehension으로도 generator 구현 가능
    - generator 이용하면 메모리 효율적 사용
    - ex. list comprehension을 사용해서 loop 돌고 싶은데 한번만 사용하면 되는 경우

In [98]:
ranger = (i for i in range(10)) 

In [99]:
for i in ranger:
    print(f'K{i}')

K0
K1
K2
K3
K4
K5
K6
K7
K8
K9


### 9-3. 재귀함수
- 자기 자신을 호출하는 함수
- 재귀가 너무 깊어지면 예외 발생(runtime error)
- text normalize할 때,
  [[a, b], [[a, b, c], b], b, c]] 
  => 모든 요소의 차원을 단일화시킬 때 
  [a, b, a, b, c,...]

In [100]:
def flatten(sent):
    for word in sent:
        if isinstance(word, list):
            # 리스트가 맞다
            for sub_word in flatten(word): # 재귀함수 사용
                yield sub_word
            # yield from flatten(word) # 위 두줄; python 3.3 이상이면 재귀함수 이렇게 사용가능 
        else:
            yield word
            
a = [1, 2, [2, 3, 4], [[[1, 2]]]]
flatten(a)

<generator object flatten at 0x000001E637885D48>

In [101]:
# 첫번째 인자의 타입을 물어보는 함수
isinstance('word', int)

False

In [102]:
for i in flatten(a):
    print(i)

1
2
2
3
4
1
2


### 9-4. 예외처리 | exception handling
- 프로그램 동작 중 예외가 발생했을 때 대처하기 위함
- 사용자에게 예외를 알리고, 원하는 조치를 설정한다.
- 프로그램이 정상적으로 종료가 될 수 있다.
- 에러가 여러개일 때는 각각의 에러에 따라 적어줌

In [9]:
try: 
    # 예외가 발생할 수도 있는 코드 블럭
    5/0
except ZeroDivisionError:
    # 예외 시 행할 행동
    print('0으로 나눌 수 없음')

0으로 나눌 수 없음


In [16]:
for i in range(10):
    try:
        print(10 / i)
    # 나누기가 안되는 부분에서만 exception
    except ZeroDivisionError:
        print('0으로 나눌 수 없음')

0으로 나눌 수 없음
10.0
5.0
3.3333333333333335
2.5
2.0
1.6666666666666667
1.4285714285714286
1.25
1.1111111111111112


#### 1) 예외의 종류

In [11]:
5 / 0

ZeroDivisionError: division by zero

In [12]:
a = [1, 2, 3]
a[5]

IndexError: list index out of range

In [13]:
int('hello')

ValueError: invalid literal for int() with base 10: 'hello'

In [14]:
k += 1

NameError: name 'k' is not defined

In [18]:
word = 'hello'

while True:
    index = input('인덱스를 입력하세요>> ')
    if index == 'q':
        break
    try: 
        index = int(index) # ValueError, IndexError
        print(word[index])
    except ValueError as e1: # 예외 핸들러
        print(e1) # 에러 메시지
    except IndexError as e2:
        print(e2)

인덱스를 입력하세요>>  1


e


인덱스를 입력하세요>>  8


string index out of range


인덱스를 입력하세요>>  h


invalid literal for int() with base 10: 'h'


인덱스를 입력하세요>>  q


#### 2) 예외 일으키기
- 프로그램을 강제 종료시키기 위해 사용함
- raise, assert(내가 원하는 조건을 보장해라! 주로 definition에서 많이 사용)
- AssertionError

In [ ]:
# 형태
raise 예외타입(메시지)

In [19]:
while True:
    num = input('숫자>> ')
    if not num.isdigit():
        raise ValueError('숫자가 아닙니다!')
    else:
        print(int(num) + 5)

숫자>>  2


7


숫자>>  n


ValueError: 숫자가 아닙니다!

In [ ]:
assert <참인 조건>, <False일 경우 내보낼 메시지>

In [20]:
# assertion으로 코드 길이 축소
def get_binary(num):
    assert isinstance(num, int), '정수 아님'
    return bin(num)

get_binary('10')

AssertionError: 정수 아님

#### 3) 사용자 정의 예외 타입
- 내가 원하는 에러(정해진 에러 외에도)
- class 선언, Exception 클래스를 상속 받는다. 

In [35]:
class MyException(Exception):
    pass

In [36]:
cities = 'dublin newyork seoul TOKYO'.split()

for city in cities:
    if city.isupper():
        raise MyException('대문자 안됨')
    else: 
        print(city)

dublin
newyork
seoul


MyException: 대문자 안됨

In [37]:
# 생성자에 접근해서 수정
class UppercaseException(Exception):
    def __init__(self):
        super().__init__('대문자 안된다구')

In [38]:
cities = 'dublin newyork seoul TOKYO'.split()

for city in cities:
    if city.isupper():
        raise UppercaseException
    else: 
        print(city)

dublin
newyork
seoul


UppercaseException: 대문자 안된다구